# Emissions mitigation widget  
Data preparation for the emissions mitigation widget.  
Data model is as follows: 
  
`indicator`: 'Reduce deforestation', 'Reforestation (Tropics)', 'Forest Management (Global)', 'Grassland and savanna fire mgmt', 'Reduce peatland degradation and conversion', 'Reduce mangrove loss', 'Mangrove Restoration'  
`category`: "Mangrove" or "Other"  
`value`: _int_    
`year`: 2020



In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import fiona

In [2]:
df = pd.read_csv('https://storage.googleapis.com/mangrove_atlas/widget_data/emissions_per_hectare.csv')
df.head()

,Country,defor_techden,refor_techden,formgmts_techden,gfire_techden,peat_techden,mangrove_techden,mangrest_techden
0,Angola,219.6426,179.0867,4.1586,18.1093,NaN,1404.4666,432.816400
1,Anguilla,136.5638,NaN,0.0000,NaN,NaN,1215.9538,5095.900000
2,Antigua & Barbuda,136.5638,NaN,0.0000,NaN,NaN,1342.5897,714.171100
3,Aruba,136.5638,NaN,0.0000,NaN,NaN,1268.6633,577.412800
4,Australia,NaN,238.2483,17.0699,5.6156,1263.144,1280.0382,436.696045


In [3]:
df_long = pd.melt(df, id_vars='Country')
df_long

,Country,variable,value
0,Angola,defor_techden,219.6426
1,Anguilla,defor_techden,136.5638
2,Antigua & Barbuda,defor_techden,136.5638
3,Aruba,defor_techden,136.5638
4,Australia,defor_techden,NaN
...,...,...,...
737,Vietnam,mangrest_techden,368.3457
738,United States Virgin Islands,mangrest_techden,505.6085
739,Yemen,mangrest_techden,756.2951
740,Zambia,mangrest_techden,NaN


In [5]:
df_long.variable.unique()

array(['defor_techden', 'refor_techden', 'formgmts_techden',
       'gfire_techden', 'peat_techden', 'mangrove_techden',
       'mangrest_techden'], dtype=object)

In [12]:
df_vars = pd.DataFrame(data={'variable':['defor_techden', 'refor_techden', 'formgmts_techden',
       'gfire_techden', 'peat_techden', 'mangrove_techden',
       'mangrest_techden'],
       'indicator':['Reduce deforestation','Reforestation (Tropics)','Forest Management (Global)','Grassland and savanna fire mgmt',
       'Reduce peatland degradation and conversion','Reduce mangrove loss','Mangrove Restoration'],
       'category': ['Other','Other','Other','Other','Other','Mangrove','Mangrove']})
df_vars

,variable,indicator,category
0,defor_techden,Reduce deforestation,Other
1,refor_techden,Reforestation (Tropics),Other
2,formgmts_techden,Forest Management (Global),Other
3,gfire_techden,Grassland and savanna fire mgmt,Other
4,peat_techden,Reduce peatland degradation and conversion,Other
5,mangrove_techden,Reduce mangrove loss,Mangrove
6,mangrest_techden,Mangrove Restoration,Mangrove


## Add location ids

In [5]:
rp = df[['Country']]
rp.head()

,Country
0,Angola
1,Anguilla
2,Antigua & Barbuda
3,Aruba
4,Australia


In [17]:
# Import gadm file to retrieve ISO codes
gadm = gpd.read_file('../../../../data/gadm36_level0_original/gadm36_level0_original.shp')
isocodes = gadm[['GID_0', 'NAME_0']].copy()
isocodes.rename(columns={'GID_0':'iso', 'NAME_0':'Country'}, inplace=True)
isocodes.head()

,iso,Country
0,ABW,Aruba
1,AFG,Afghanistan
2,AGO,Angola
3,AIA,Anguilla
4,ALA,Åland


In [8]:
# Import locations to get staging ids
#locations = pd.read_csv('../../../data/staging_locations.csv')
#locations = locations[locations['location_type'] == 'country']
#locations = locations[['id', 'name', 'iso']].copy()
#locations.head()

In [25]:
locations_file = 'https://storage.googleapis.com/mangrove_atlas/boundaries/processed/location_final/locations_v3_not_merged_with_old.gpkg'
locations = gpd.read_file(locations_file)
locations = locations[locations['type'] == 'country'][['name','iso', 'location_idn']]
locations.head()

,name,iso,location_idn
82,Qatar,QAT,06d2e6f9-bc89-59bf-a0e2-ab804e5db9fd
89,Mayotte,MYT,0750953f-4af9-549b-aeea-329663249a56
118,Vietnam,VNM,09a1ab14-11ad-56ec-8acb-a149e5697abd
132,Grenada,GRD,0b0ecb56-bb8e-5ef1-b8ee-3cdad67fed0e
149,India,IND,0c07ca53-7b17-5650-a2c6-0cc27249a4bd


In [11]:
api_locs = pd.read_csv('https://storage.googleapis.com/mangrove_atlas/widget_data/locations_staging.csv')
api_locs.rename(columns={'location_id': 'location_idn'}, inplace=True)
api_locs.head()

,id,location_idn
0,1563,000bd204-c0fd-510b-a1ad-132a7ef7470d
1,1564,00250a0f-f66d-54a0-b7a3-d80035881cbf
2,1565,0041637b-f6a2-5b89-87ce-850f5c5431b3
3,1566,005b49ef-6b7f-575a-85b3-ff19261a0755
4,1567,00921349-70fb-5a7e-8207-b3157aecc349


In [21]:
#Add ISO codes
df_final = pd.merge(df_long, isocodes, on = 'Country', how='left')
df_final.head()

,Country,variable,value,iso
0,Angola,defor_techden,219.6426,AGO
1,Anguilla,defor_techden,136.5638,AIA
2,Antigua & Barbuda,defor_techden,136.5638,NaN
3,Aruba,defor_techden,136.5638,ABW
4,Australia,defor_techden,NaN,AUS


In [23]:
#Check countries without assigned ISO
df_final[df_final['iso'].isnull()]['Country'].unique()

array(['Antigua & Barbuda', 'The Bahamas',
       'Bonaire, Sint-Eustasius, Saba', 'Virgin Islands, British',
       'Cayman Is.', 'Congo, DRC', "Cote d'Ivoire", 'The Gambia',
       'St. Kitts & Nevis', 'St. Lucia', 'Saint Martin',
       'Trinidad & Tobago', 'Turks & Caicos Is.',
       'United States Virgin Islands'], dtype=object)

In [40]:
cnty = 'Lucia'
locations[locations['name'].str.contains(cnty)]

,name,iso,location_idn
2108,Saint Lucia,LCA,aa34d72a-c6b6-58dd-ba0e-2343f49e3f25


In [ ]:
df_final[df_final['Antigua & Barbuda']]['iso'] == 'ATG'
df_final[df_final['The Bahamas']]['iso'] == 'BHS'
df_final[df_final['Bonaire, Sint-Eustasius, Saba']]['iso'] == 'BES'
df_final[df_final['Virgin Islands, British']]['iso'] == 'VGB'
df_final[df_final['Cayman Is.']]['iso'] == 'CYM'
df_final[df_final['Congo, DRC']]['iso'] == 'COD'
df_final[df_final["Cote d'Ivoire"]]['iso'] == 'CIV'
df_final[df_final['The Gambia']]['iso'] == 'GMB'
df_final[df_final['St. Kitts & Nevis']]['iso'] == 'KNA'
df_final[df_final['St. Lucia']]['iso'] == 'LCA'
df_final[df_final['Antigua & Barbuda']]['iso'] == 'ATG'
df_final[df_final['Antigua & Barbuda']]['iso'] == 'ATG'
df_final[df_final['Antigua & Barbuda']]['iso'] == 'ATG'
df_final[df_final['Antigua & Barbuda']]['iso'] == 'ATG'
df_final[df_final['Antigua & Barbuda']]['iso'] == 'ATG'



In [24]:
locations

,iso,location_idn
82,QAT,06d2e6f9-bc89-59bf-a0e2-ab804e5db9fd
89,MYT,0750953f-4af9-549b-aeea-329663249a56
118,VNM,09a1ab14-11ad-56ec-8acb-a149e5697abd
132,GRD,0b0ecb56-bb8e-5ef1-b8ee-3cdad67fed0e
149,IND,0c07ca53-7b17-5650-a2c6-0cc27249a4bd
...,...,...
2996,BHR,f309afe5-27b5-575a-aa2c-7598a53dffa4
3024,BGD,f58d8175-a251-50e8-b1e1-756d60d70d96
3053,MMR,f848599b-9858-59d4-afe3-02c21321630e
3062,SXM,f91064fe-8030-5230-b71a-4b486d80b1e4


In [20]:

df_final = df_final.merge(locations, on='iso', how='left')
#mang_df_final = pd.merge(mang_df_final, api_locs, on='location_idn', how='left')
#mang_df_final = mang_df_final[mang_df_final.id.notna()]
df_final

,Country,variable,value,iso,location_idn
0,Angola,defor_techden,219.6426,AGO,27ceab8c-946e-5286-a06f-8bd98ec81f77
1,Anguilla,defor_techden,136.5638,AIA,1ce4c2e5-8456-5db8-8e34-8bfe86083790
2,Antigua & Barbuda,defor_techden,136.5638,NaN,NaN
3,Aruba,defor_techden,136.5638,ABW,5d1bd36b-f200-593d-972a-bf69abdd912f
4,Australia,defor_techden,NaN,AUS,48287653-09c8-5cfd-95b8-6a5b66b600bb
...,...,...,...,...,...
737,Vietnam,mangrest_techden,368.3457,VNM,09a1ab14-11ad-56ec-8acb-a149e5697abd
738,United States Virgin Islands,mangrest_techden,505.6085,NaN,NaN
739,Yemen,mangrest_techden,756.2951,YEM,5aff671b-1089-5020-b688-8bc2e4a60e34
740,Zambia,mangrest_techden,NaN,ZMB,NaN


In [27]:
country_iso = pd.merge(rp, isocodes, how='left', left_on='Country', right_on='NAME_0')
country_iso.rename(columns={'GID_0': 'iso'}, inplace=True)
country_iso.head()

,Country,iso,NAME_0
0,Angola,AGO,Angola
1,Anguilla,AIA,Anguilla
2,Antigua & Barbuda,NaN,NaN
3,Aruba,ABW,Aruba
4,Australia,AUS,Australia


In [31]:
country_iso_locid = pd.merge(country_iso, locations, how='left', on='iso').drop(columns=['NAME_0', 'name'])
country_iso_locid

,Country,iso,id
0,Angola,AGO,1398.0
1,Anguilla,AIA,NaN
2,Antigua & Barbuda,NaN,NaN
3,Aruba,ABW,NaN
4,Australia,AUS,1399.0
...,...,...,...
101,Vietnam,VNM,1364.0
102,United States Virgin Islands,NaN,NaN
103,Yemen,YEM,1366.0
104,Zambia,ZMB,NaN


## Final format

In [34]:
df_complete = pd.merge(df_long, country_iso_locid, how='left', on='Country').drop(columns=['Country', 'iso'])
df_complete['unit'] = 'tCO2eq'
df_complete.rename(columns={'variable': 'indicator', 'id': 'location_id'}, inplace=True)
df_complete = df_complete[df_complete['location_id'].isna() == False]
df_complete

,indicator,value,location_id,unit
0,defor_techden,219.6426,1398.0,tCO2eq
4,defor_techden,NaN,1399.0,tCO2eq
6,defor_techden,NaN,1374.0,tCO2eq
7,defor_techden,496.2587,1373.0,tCO2eq
8,defor_techden,443.8740,1376.0,tCO2eq
...,...,...,...,...
734,mangrest_techden,664.8271,1324.0,tCO2eq
735,mangrest_techden,646.0532,1394.0,tCO2eq
736,mangrest_techden,444.5305,1362.0,tCO2eq
737,mangrest_techden,368.3457,1364.0,tCO2eq


In [35]:
df_complete.to_csv('../../../data/mitigation_emission_data.csv', index=False)